In [1]:
from transformers import BertTokenizer
from razdel import sentenize
from models.model_builder import AbsSummarizer
import torch
import numpy as np
import pandas as pd

In [2]:
class BertData:
    def __init__(self, bert_model, lower, max_src_tokens, max_tgt_tokens):
        self.max_src_tokens = max_src_tokens
        self.max_tgt_tokens = max_tgt_tokens
        self.tokenizer = BertTokenizer.from_pretrained(bert_model, do_lower_case=lower, do_basic_tokenize=False)
        self.sep_token = '[SEP]'
        self.cls_token = '[CLS]'
        self.pad_token = '[PAD]'
        self.tgt_bos = '[unused1] '
        self.tgt_eos = ' [unused2]'
        self.tgt_sent_split = ' [unused3] '
        self.sep_vid = self.tokenizer.vocab[self.sep_token]
        self.cls_vid = self.tokenizer.vocab[self.cls_token]
        self.pad_vid = self.tokenizer.vocab[self.pad_token]

    def preprocess(self, src, tgt):
        src_txt = [' '.join(s) for s in src]
        text = ' {} {} '.format(self.sep_token, self.cls_token).join(src_txt)
        src_tokens = self.tokenizer.tokenize(text)[:self.max_src_tokens]
        src_tokens.insert(0, self.cls_token)
        src_tokens.append(self.sep_token)
        src_indices = self.tokenizer.convert_tokens_to_ids(src_tokens)

        _segs = [-1] + [i for i, t in enumerate(src_indices) if t == self.sep_vid]
        segs = [_segs[i] - _segs[i - 1] for i in range(1, len(_segs))]
        segments_ids = []
        for i, s in enumerate(segs):
            if i % 2 == 0:
                segments_ids += s * [0]
            else:
                segments_ids += s * [1]

        return src_indices, segments_ids

In [3]:
def doc2bert(text):
    src = [s.text.lower().split() for s in sentenize(text)]
    src_indices, segments_ids = bert_data.preprocess(src, '')
    return { "src": src_indices, "segs": segments_ids }

def doc2vec(text, model, mode='MeanSum'):
    doc_bert = doc2bert(text)
    
    src = torch.tensor([doc_bert['src']])
    segs = torch.tensor([doc_bert['segs']])
    mask_src = ~(src == 0)
    
    output = model.bert(src, segs, mask_src)
    
    if mode == 'FirstCLS':
        return output[0][0]
    elif mode == 'MeanSum':
        return output[0].mean(0)
    else:
        raise Exception('Wrong mode')

In [4]:
checkpoint = torch.load('/data/alolbuhtijarov/rubert_cased_L-12_H-768_A-12_pt/model_step_15000.pt',
                        map_location=lambda storage, loc: storage)

In [5]:
args = lambda a: b

args.model_path = '/data/alolbuhtijarov/rubert_cased_L-12_H-768_A-12_pt'
args.large = False
args.temp_dir = 'temp'
args.finetune_bert = False
args.encoder = 'bert'
args.max_pos = 256
args.dec_layers = 6
args.share_emb = False
args.dec_hidden_size = 768
args.dec_heads = 8
args.dec_ff_size = 2048
args.dec_dropout = 0.2
args.use_bert_emb = False

bert_data = BertData(args.model_path, True, 510, 128)

In [6]:
model = AbsSummarizer(args, 'cpu', checkpoint)
model.eval()

AbsSummarizer(
  (bert): Bert(
    (model): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(119547, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): LayerNorm(torch.Size([768]), eps=1e-12, elementwise_affine=True)
        (dropout): Dropout(p=0.1)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): LayerNorm(t

### Clustering

In [7]:
import tqdm
from sklearn.cluster import AgglomerativeClustering
from collections import defaultdict
import csv
import json

In [8]:
def read_markup(file_name):
    with open(file_name, "r") as r:
        reader = csv.reader(r, delimiter='\t', quotechar='"')
        header = next(reader)
        for row in reader:
            assert len(header) == len(row)
            record = dict(zip(header, row))
            yield record
            

In [9]:
from sklearn.metrics import classification_report

def calc_metrics(gold_markup, url2label, url2record, output_dict=False):
    not_found_count = 0
    for first_url, second_url in list(gold_markup.keys()):
        not_found_in_labels = first_url not in url2label or second_url not in url2label
        not_found_in_records = first_url not in url2record or second_url not in url2record
        if not_found_in_labels or not_found_in_records:
            not_found_count += 1
            gold_markuo.pop((first_url, second_url))

    targets = []
    predictions = []
    for (first_url, second_url), target in gold_markup.items():
        prediction = int(url2label[first_url] == url2label[second_url])
        first = url2record.get(first_url)
        second = url2record.get(second_url)
        targets.append(target)
        predictions.append(prediction)
    return classification_report(targets, predictions, output_dict=output_dict)

In [10]:
markup_path = "/data/alolbuhtijarov/datasets/ru_threads_target.tsv"
clustering_data_path = "/data/alolbuhtijarov/datasets/ru_clustering_data.jsonl"

In [11]:
markup = defaultdict(dict)
for record in read_markup(markup_path):
    first_url = record["INPUT:first_url"]
    second_url = record["INPUT:second_url"]
    quality = int(record["OUTPUT:quality"] == "OK")
    markup[(first_url, second_url)] = quality

In [12]:
url2record = dict()
filename2url = dict()
with open(clustering_data_path, "r") as r:
    for line in r:
        record = json.loads(line)
        url2record[record["url"]] = record
        filename2url[record["file_name"]] = record["url"]

In [14]:
embeds = np.zeros((len(url2record), 768))

for i, (url, record) in tqdm.tqdm(enumerate(url2record.items()), total=embeds.shape[0]):
    text = record["title"] + ' ' + record["text"]
    text = text.lower().replace('\xa0', ' ')
    embeds[i] = doc2vec(text, model, mode='FirstCLS')


  2%|▏         | 256/10730 [00:26<14:11, 12.30it/s]


  5%|▍         | 532/10730 [00:54<15:40, 10.84it/s]


  7%|▋         | 799/10730 [01:21<19:02,  8.69it/s]


 10%|█         | 1082/10730 [01:49<16:21,  9.83it/s]


 13%|█▎        | 1356/10730 [02:16<19:54,  7.85it/s]


...

 91%|█████████▏| 9801/10730 [17:15<01:59,  7.78it/s]


 93%|█████████▎| 10029/10730 [17:42<01:40,  7.00it/s]


 96%|█████████▌| 10267/10730 [18:08<01:17,  5.98it/s]


 98%|█████████▊| 10524/10730 [18:35<00:22,  9.18it/s]


100%|██████████| 10730/10730 [18:57<00:00,  9.43it/s]


In [15]:
clustering_model = AgglomerativeClustering(n_clusters=None,
                                           distance_threshold=0.2476,
                                           linkage="single",
                                           affinity="cosine")

clustering_model.fit(embeds)
labels = clustering_model.labels_

id2url = dict()
for i, (url, _) in enumerate(url2record.items()):
    id2url[i] = url

url2label = dict()
for i, label in enumerate(labels):
    url2label[id2url[i]] = label

In [22]:
with open('bad.txt', 'w') as f:
    targets = []
    predictions = []
    for (first_url, second_url), target in markup.items():
        prediction = int(url2label[first_url] == url2label[second_url])
        first = url2record.get(first_url)
        second = url2record.get(second_url)
        targets.append(target)
        predictions.append(prediction)
        if target != prediction:
            f.write(first_url + '\n')
            f.write(second_url + '\n')
            f.write(url2record.get(first_url)['title'] + '\n')
            f.write(url2record.get(second_url)['title'] + '\n')
            f.write(f'target = {target}\nprediction = {prediction}\n')
            f.write(30*'-' + '\n')

'26 новых случаев коронавируса выявили в Мытищах'

In [ ]:
def get_quality(dist_threshold, print_result=False):
    clustering_model = AgglomerativeClustering(n_clusters=None,
                                           distance_threshold=dist_threshold,
                                           linkage="single",
                                           affinity="cosine")

    clustering_model.fit(embeds)
    labels = clustering_model.labels_
    
    id2url = dict()
    for i, (url, _) in enumerate(url2record.items()):
        id2url[i] = url

    url2label = dict()
    for i, label in enumerate(labels):
        url2label[id2url[i]] = label
        
    if print_result:
        print(calc_metrics(markup, url2label, url2record))
        return
    metrics = calc_metrics(markup, url2label, url2record, output_dict=True)
    return metrics['macro avg']['f1-score']

In [ ]:
domain = np.logspace(-5, 0, 70)
quals = [get_quality(dist) for dist in tqdm.tqdm(domain, total=70)]

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(domain, quals);

In [ ]:
closer_domain = np.linspace(domain[np.argmax(quals)-5], domain[np.argmax(quals)+5], 70)
closer_quals = [get_quality(dist) for dist in tqdm.tqdm(closer_domain, total=70)]

In [ ]:
plt.plot(closer_domain, closer_quals);

In [ ]:
best_dist = closer_domain[np.argmax(closer_quals)]

In [ ]:
best_dist

In [60]:
get_quality(best_dist, print_result=True)

              precision    recall  f1-score   support

           0       0.88      0.82      0.85      1571
           1       0.77      0.85      0.81      1130

    accuracy                           0.83      2701
   macro avg       0.83      0.83      0.83      2701
weighted avg       0.84      0.83      0.83      2701



### Result

40k checkpoint; title + ' ' + text; MeanSum, dist = 0.187

In [38]:
get_quality(best_dist, print_result=True)

              precision    recall  f1-score   support

           0       0.87      0.84      0.85      1571
           1       0.78      0.83      0.81      1130

    accuracy                           0.83      2701
   macro avg       0.83      0.83      0.83      2701
weighted avg       0.84      0.83      0.84      2701

